In [67]:
from __future__ import annotations
import polars as pl

dirty = pl.read_csv("data/GCM_Total.res", separator="\t", truncate_ragged_lines=True)

ds = pl.DataFrame({col: [] for col in dirty.columns})

new_columns = []
for i, col in enumerate(ds.columns):
    if "duplicated" in col:
        new_col = f"{ds.columns[i-1]}_class"
    elif col == '':
        new_col = f"{ds.columns[i-1]}_class"
    else:
        new_col = col
    new_columns.append(new_col)

ds.columns = new_columns

print(ds.columns)

['Description', 'Accession', 'Tumor__Breast_Adeno_09-B_003A', 'Tumor__Breast_Adeno_09-B_003A_class', 'Tumor__Breast_Adeno_09-B_005A', 'Tumor__Breast_Adeno_09-B_005A_class', 'Tumor__Breast_Adeno_92_I_078', 'Tumor__Breast_Adeno_92_I_078_class', 'Tumor__Breast_Adeno_93_I_192', 'Tumor__Breast_Adeno_93_I_192_class', 'Tumor__Breast_Adeno_93_I_250', 'Tumor__Breast_Adeno_93_I_250_class', 'Tumor__Breast_Adeno_94_I_155', 'Tumor__Breast_Adeno_94_I_155_class', 'Tumor__Breast_Adeno_94_I_159', 'Tumor__Breast_Adeno_94_I_159_class', 'Tumor__Breast_Adeno_95_I_029', 'Tumor__Breast_Adeno_95_I_029_class', 'Tumor__Breast_Adeno_9912c068_CC', 'Tumor__Breast_Adeno_9912c068_CC_class', 'Tumor__Breast_Adeno_mBRT1_(8697)', 'Tumor__Breast_Adeno_mBRT1_(8697)_class', 'Tumor__Breast_Adeno_mBRT2_(9078)', 'Tumor__Breast_Adeno_mBRT2_(9078)_class', 'Tumor__Prostate_Adeno_94_I_052', 'Tumor__Prostate_Adeno_94_I_052_class', 'Tumor__Prostate_Adeno_95_I_249', 'Tumor__Prostate_Adeno_95_I_249_class', 'Tumor__Prostate_Adeno_95_I

In [68]:
# remove columns without names
cols = ds.columns
print(f"Total columns: {len(cols)}")

a = 0
for col in cols:
    print(f"{a}: {col}")

Total columns: 563
0: Description
0: Accession
0: Tumor__Breast_Adeno_09-B_003A
0: Tumor__Breast_Adeno_09-B_003A_class
0: Tumor__Breast_Adeno_09-B_005A
0: Tumor__Breast_Adeno_09-B_005A_class
0: Tumor__Breast_Adeno_92_I_078
0: Tumor__Breast_Adeno_92_I_078_class
0: Tumor__Breast_Adeno_93_I_192
0: Tumor__Breast_Adeno_93_I_192_class
0: Tumor__Breast_Adeno_93_I_250
0: Tumor__Breast_Adeno_93_I_250_class
0: Tumor__Breast_Adeno_94_I_155
0: Tumor__Breast_Adeno_94_I_155_class
0: Tumor__Breast_Adeno_94_I_159
0: Tumor__Breast_Adeno_94_I_159_class
0: Tumor__Breast_Adeno_95_I_029
0: Tumor__Breast_Adeno_95_I_029_class
0: Tumor__Breast_Adeno_9912c068_CC
0: Tumor__Breast_Adeno_9912c068_CC_class
0: Tumor__Breast_Adeno_mBRT1_(8697)
0: Tumor__Breast_Adeno_mBRT1_(8697)_class
0: Tumor__Breast_Adeno_mBRT2_(9078)
0: Tumor__Breast_Adeno_mBRT2_(9078)_class
0: Tumor__Prostate_Adeno_94_I_052
0: Tumor__Prostate_Adeno_94_I_052_class
0: Tumor__Prostate_Adeno_95_I_249
0: Tumor__Prostate_Adeno_95_I_249_class
0: Tumor_